# Object Detection with Detectron2 - Training

## Import Required modules and Register the dataset
The following cell imports all the required modules, frameworks which are involved in training the model.

In [1]:
import torch, torchvision

import detectron2
from detectron2.utils.logger import setup_logger
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo

/home/studio-lab-user/.conda/envs/object-detection-gpu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The following cell initializes Detectron2's logger and register the dataset we are using.

In [2]:
setup_logger()
register_coco_instances("car", {}, "../data/data/labels/label_train_coco.json", "../data/data/images/train")

## Detectron2's model zoo
We use `Transfer learning` in training the our model to detect cars. In this case, we will be using the `Mask R-CNN ResNet-50 FPN 3x` model pre-trained for COCO instance segmentation.<br>
Benefits of using Transfer Learning:
- Saves time
- Saves Computational resources of training a model from the scratch.

Detectron2's model zoo, check out: https://github.com/facebookresearch/detectron2/blob/main/MODEL_ZOO.md


In [3]:
MODEL = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"

## Detectron2 Object Detection Configuration:
This cell configures parameters and file paths for training a Detectron2 object detection model on a custom dataset and detect the object [car]

In [12]:
PATH_IMAGES = "../data/data/data/images/train"
PATH_LABELS = "../data/data/labels/labels_train_coco.json"
PATH_OUTPUT = "../output"

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(MODEL))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(MODEL)
cfg.DATASETS.TRAIN = ("car")
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 4
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.01
cfg.SOLVER.MAX_ITER = 600
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.OUTPUT_DIR = PATH_OUTPUT

## Training the Model
The following cell starts the training and check if there is a previous checkpoint from a partially trained model. if yes,then  it will resume the training rather than starting from scratch.

In [13]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

[12/17 21:11:56 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res